In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from evaluation import *
import seaborn as sns

### Load Models

In [ ]:
variant = 'ViT-L-14'
ckpt_path = "./logs/_2022_12_13-18_19_55-model_ViT-L-14-lr_0.0001-b_56-j_8-p_amp_bf16/checkpoints/epoch_3.pt"

In [ ]:
df = evaluate_model(variant, 'openai', ckpt_path, alphas=[0.5, 0.7], device=2, batch_size=256)

### Visualise Results

In [ ]:
files = get_files("/home/synopsis/git/open_clip/src/val_logs/", extensions=[".csv"])
df = pd.concat([pd.read_csv(f) for f in files])
df = df.reset_index(drop=True)
df["Accuracy"] = df["Accuracy"].apply(lambda x: round(100*x))
df = df[df.Alpha.isin([0.5, 1.0])]

df["Experiment"] = df.Experiment.replace(
    {'Last-5-Unfrozen': 'Last-5-Unfrozen_Epoch-3',
     'Fully-Unfrozen': 'Fully-Unfrozen__Thresh-0.9__Epoch-3'
    }
)

In [ ]:
sorted(files)

In [ ]:
sorted(df.Experiment.unique())

In [ ]:
exps = [
    'Fully-Unfrozen__No-Thresh__Epoch-4_metrics.csv',
    'Fully-Unfrozen__Thresh-0.9__Epoch-3',
]
filt = df["Experiment"].isin(exps)
sub = df[filt]
sub = sub[sub.Alpha == 0.5]
sub = sub.sort_values(["Experiment", "Categories"])
sub

In [ ]:
# exp = "Last-3-Unfrozen_Epoch-3_metrics.csv"
# ss = df[df.Experiment == exp]

plt.rcParams["figure.figsize"] = [25, 10]
ax=sns.barplot(
    x="Categories", y="Accuracy", hue="Experiment", data=sub, ci=0,
)
_= [ax.bar_label(c) for c in ax.containers]
_= ax.set(title = f"Dataset With / Without Thresholding @Alpha=0.5")
_= plt.xticks(rotation=90)

In [ ]:
plt.rcParams["figure.figsize"] = [20, 10]
g = sns.FacetGrid(sub, row="Alpha", height=10, aspect=2)
g = g.map(sns.barplot, "Categories", "Accuracy", "Experiment", palette="crest")
_= plt.xticks(rotation=90)